<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/Graded_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275 - Advanced Supply Chain Systems Planning and Network Design
# **Graded Assignment 4**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data

!pip install scgraph==2.1.0         # Python package used to compute paths and distances on a real-world transportation network
!pip install scgraph_data==2.0.0    # Python package used to compute paths and distances on a real-world transportation network



In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


import scgraph                                                  # For computing paths and distances on a real-world transportation network
from scgraph.geographs.us_freeway import us_freeway_geograph    # Data on US highway network (for road paths and distances)
from scgraph.geographs.marnet import marnet_geograph            # Data on maritime routes (for ocean paths and distances)
import matplotlib.pyplot as plt                                 # Plotting library

## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               color,                       # Color of the marker icon
               background_color,            # Background color of the marker icon
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=color,
                border_color=color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing the shortest path between two points on a real road or ocean network**

In [ ]:
# Function for computing the shortest path between two points on a real road or ocean network

def shortest_path(origin, destination, mode, result, unit='mi'):

    # Extract coordinates from origin and destination objects
    origin_coordinates = (origin.lat, origin.lon)
    destination_coordinates = (destination.lat, destination.lon)

    # Calculate the shortest path on the ocean network
    if mode == 'ocean':
        output = marnet_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Calculate the shortest path on the road network
    elif mode == 'road':
        output = us_freeway_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Return the total distance of the path
    if result == 'distance':
        return output['length']

    # Return the coordinates representing the path
    elif result == 'coordinate_path':
        return output['coordinate_path']


### **Ploting flows (real network) on the map**

In [ ]:
# Functions that adjust the arc path to ensure longitude continuity across the globe

def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Plots real flow_vars on a Folium map based on optimization model results

def plot_real_flows(map,              # Folium map object where flows will be plotted.
               vars,                  # Dictionary of flow decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               mode,                  # Transportation mode (road or ocean)
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flow_vars, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flow_vars
        if var.X > 0:

            # Get the shortest path coordinates for the node pair
            path = shortest_path(nodes[node1_key], nodes[node2_key], mode = mode, result = 'coordinate_path')

            # Add the path as a polyline on the map, with width proportional to the flow value
            folium.PolyLine(getCleanArcPath(path),
                      color=color,
                      weight=var.X / max_val * max_width,   # Normalize weight by maximum flow value
                      opacity=opacity).add_to(map)


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing distribution center data
dc_file ='https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/mo_pset/distribution_centers.csv'

# Loading DC data into a pandas DataFrame
distribution_centers_df = pd.read_csv(dc_file)

# Displaying the first few rows of the DataFrame to verify the data
distribution_centers_df.head()


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/mo_pset/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

In [ ]:
# File containing port data
port_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/mo_pset/ports.csv'

# Loading port data into a pandas DataFrame
ports_df = pd.read_csv(port_data_file)

# Displaying the first few rows of the DataFrame to verify the data
ports_df.head()


#### Definition of Classes

In [ ]:
# Class representing a DistributionCenter object

class DistributionCenter():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # DistributionCenter's ID
        self.name = name          # DistributionCenter's name
        self.lat = lat            # DistributionCenter's latitude
        self.lon = lon            # DistributionCenter's longitude
        self.demand = demand      # DistributionCenter's demand


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, country, supply, purchase_price):
        self.ID = ID                              # Supplier's ID
        self.name = name                          # Supplier's name
        self.lat = lat                            # Supplier's latitude
        self.lon = lon                            # Supplier's longitude
        self.country = country                    # Supplier's country
        self.supply = supply                      # Supplier's available supply
        self.purchase_price = purchase_price      # Supplier's purchase price


In [ ]:
# Class representing a Port object

class Port():
    def __init__(self, ID, name, lat, lon, dwell_time):
        self.ID = ID                  # Port's ID
        self.name = name              # Ports's name
        self.lat = lat                # Ports's latitude
        self.lon = lon                # Ports's longitude
        self.dwell_time = dwell_time  # Ports's dwell time


#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of DC objects
distribution_centers = dict()
for i, row in distribution_centers_df.iterrows():
    distribution_centers[row['ID']] = DistributionCenter(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # DistributionCenter's name
                                    lat=row['lat'],         # DistributionCenter's latitude
                                    lon=row['lon'],         # DistributionCenter's longitude
                                    demand=row['demand'])   # DistributionCenter's demand

# Merging the DC dictionary into the existing nodes dictionary
nodes = {**nodes, **distribution_centers}

In [ ]:
# Computing total demand
total_demand = sum([d.demand for d in distribution_centers.values()])

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],                             # Supplier's ID
                                    name=row['name'],                         # Supplier's name
                                    lat=row['lat'],                           # Supplier's latitude
                                    lon=row['lon'],                           # Supplier's longitude
                                    country=row['country'],                   # Supplier's country
                                    purchase_price=row['purchase_price'],     # Supplier's purchase price
                                    supply=row['supply'])                     # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **suppliers}

In [ ]:
# Differentiating between US and international suppliers

US_suppliers = {s: supplier for s, supplier in suppliers.items() if supplier.country == 'US'}
INTL_suppliers = {s: supplier for s, supplier in suppliers.items() if supplier.country != 'US'}

In [ ]:
# Creating a dictionary of port objects
ports = dict()
for i, row in ports_df.iterrows():
    ports[row['ID']] = Port(ID = row['ID'],                         # Port's ID
                                    name = row['name'],             # Port's name
                                    lat = row['lat'],               # Port's latitude
                                    lon = row['lon'],               # Port's longitude
                                    dwell_time = row['dwell_time']) # Port's longitude

# Merging the port dictionary into the existing nodes dictionary
nodes = {**nodes, **ports}

#### Visualizing node objects

In [ ]:
# Create a new map
map = folium.Map([40, -100.0], zoom_start=2)

# Plot DC locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=distribution_centers, icon='warehouse', color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary to store distances between suppliers and ports, and ports and customers

distances = dict()  # Initialize an empty dictionary to store distances

# Calculate distances between each international supplier and port using the ocean network
for s, supplier in INTL_suppliers.items():
    for p, port in ports.items():
        distances[s, p] = shortest_path(origin=supplier, destination=port, mode='ocean', result='distance', unit='km')

# Calculate distances between each port and distribution center using the road network
for p, port in ports.items():
  for d, distribution_center in distribution_centers.items():
        distances[p, d] = shortest_path(origin=port, destination=distribution_center, mode='road', result='distance', unit='km')

# Calculate distances between each US supplier and distribution center using the road network
for s, supplier in US_suppliers.items():
    for d, distribution_center in distribution_centers.items():
      distances[s, d] = shortest_path(origin = supplier, destination = distribution_center, mode = 'road', result = 'distance', unit = 'km')



#### Arc costs

In [ ]:
# Creating a dictionary to store unit transportation costs between different nodes
unit_cost = dict()

# Unit transportation costs beteween suppliers and ports (ocean)

# Cost per kilometer for ocean transport
cost_km_ocean = 0.01

# Calculate the ocean transport cost for each supplier-port pair
for s, supplier in INTL_suppliers.items():
    for p, port in ports.items():
        # Ocean transport cost is calculated by multiplying distance by cost per kilometer
        unit_cost[s, p] = distances[s, p] * cost_km_ocean


# Unit transportation costs beteween ports and customers (road)

# Minimum transportation cost for short distances
min_cost = 25

# Distance threshold to differentiate between long and short distance rates
distance_threshold = 1500

# Cost per kilometer for long-distance road transport
long_distance_rate = 0.1

# Cost per kilometer for short-distance road transport
short_distance_rate = 0.2

# Calculate the road transport cost for each port-distribution center pair
for p, port in ports.items():
    for d, distribution_center in distribution_centers.items():
        # Apply long-distance rate if the distance exceeds the threshold
        if distances[p, d] > distance_threshold:
            unit_cost[p, d] = distances[p, d] * long_distance_rate
        # Apply short-distance rate, ensuring cost does not fall below minimum cost
        else:
            unit_cost[p, d] = max(min_cost, distances[p, d] * short_distance_rate)

# Calculate the road transport cost for each US supplier-distribution center pair
for s, supplier in US_suppliers.items():
    for d, distribution_center in distribution_centers.items():
        # Apply long-distance rate if the distance exceeds the threshold
        if distances[s, d] > distance_threshold:
            unit_cost[s, d] = distances[s, d] * long_distance_rate
        # Apply short-distance rate, ensuring cost does not fall below minimum cost
        else:
            unit_cost[s, d] = max(min_cost, distances[s, d] * short_distance_rate)


#### Arc lead times

In [ ]:
# Defining parameters
ocean_speed = 750  # per day
road_speed = 400  # per day

# Lead time dictionary
lead_time = dict()

# Compute ocean lead times
for s, supplier in INTL_suppliers.items():
    for p, port in ports.items():
        lead_time[s, p] = round(distances[s, p] / ocean_speed, 2) + port.dwell_time

# Compute road lead times
for p, port in ports.items():
    for d, distribution_center in distribution_centers.items():
        lead_time[p, d] = round(distances[p, d] / road_speed, 2)

for s, supplier in US_suppliers.items():
    for d, distribution_center in distribution_centers.items():
        lead_time[s, d] = round(distances[s, d] / road_speed, 2)


#### Arc emission

In [ ]:
# Defining the emission factors (in pounds of CO2 per ton-mile)

# Source:
# Shirley, C., & Gecan, R. (2022). Emissions of Carbon Dioxide in the Transportation Sector.
# https://www.cbo.gov/system/files/2022-12/58566-co2-emissions-transportation.pdf

# Defining emission factors in tons of CO2
ton_to_pounds = 2204.62
emission_fact_road = 0.4 / ton_to_pounds
emission_fact_ocean = 0.14 / ton_to_pounds

# Emissions dictionary
unit_emissions = dict()

# Compute ocean emissions directly
for s, supplier in INTL_suppliers.items():
    for p, port in ports.items():
        unit_emissions[s, p] = distances[s, p] * emission_fact_ocean

# Compute road emissions directly
for p, port in ports.items():
    for d, distribution_center in distribution_centers.items():
        unit_emissions[p, d] = distances[p, d] * emission_fact_road

for s, supplier in US_suppliers.items():
    for d, distribution_center in distribution_centers.items():
        unit_emissions[s, d] = distances[s, d] * emission_fact_road


## **Optimization model**

### **Creating the optimization model components**

*Note: In this first block of code, we are creating the components of the optimization model, including decision variables, expressions for the objective function, and constraints. We are not yet specifying the objective function, which will be defined in a later step (see below). We do this to avoid rewriting the same code multiple times*

In [ ]:
e = grb.Env("gurobi.log")                      # Initialize the Gurobi environment, logging to "gurobi.log"
e.setParam('OutputFlag', 0)                    # Suppress Gurobi output for a cleaner log
# model.Params.MIPGap = 0.000001               # Uncomment to set a tighter MIP gap if needed for precise solutions

# Initializing the model
model = grb.Model("Model", env=e)              # Create the Gurobi model for solving the multi-objective optimization problem

# Creating Decision Variables
flow_vars_ocean = dict()                       # Decision variables for ocean transport flows
flow_vars_road = dict()                        # Decision variables for road transport flows

# Define ocean transport flow variables between international suppliers and ports
for s in INTL_suppliers:
    for p in ports:
        flow_vars_ocean[s, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                   name="flow_{0}_{1}".format(s, p))

# Define road transport flow variables between ports and distribution centers
for p in ports:
    for d in distribution_centers:
        flow_vars_road[p, d] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                   name="flow_{0}_{1}".format(p, d))

# Define road transport flow variables between U.S. suppliers and distribution centers
for s in US_suppliers:
    for d in distribution_centers:
        flow_vars_road[s, d] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                   name="flow_{0}_{1}".format(s, d))

# Creating the objective function components

# Calculate total transport cost
transport_cost = grb.quicksum(unit_cost[s, p] * flow_vars_ocean[s, p] for s in INTL_suppliers for p in ports) + \
                 grb.quicksum(unit_cost[p, d] * flow_vars_road[p, d] for p in ports for d in distribution_centers) + \
                 grb.quicksum(unit_cost[s, d] * flow_vars_road[s, d] for s in US_suppliers for d in distribution_centers)

# Calculate total purchase cost
purchase_cost = grb.quicksum(suppliers[s].purchase_price * flow_vars_ocean[s, p] for s in INTL_suppliers for p in ports) + \
                grb.quicksum(suppliers[s].purchase_price * flow_vars_road[s, d] for s in US_suppliers for d in distribution_centers)

# Total cost includes both transport and purchase costs
total_cost = transport_cost + purchase_cost

# Calculate total emissions from transport
total_emissions = grb.quicksum(unit_emissions[s, p] * flow_vars_ocean[s, p] for s in INTL_suppliers for p in ports) + \
                  grb.quicksum(unit_emissions[p, d] * flow_vars_road[p, d] for p in ports for d in distribution_centers) + \
                  grb.quicksum(unit_emissions[s, d] * flow_vars_road[s, d] for s in US_suppliers for d in distribution_centers)

# Calculate total lead time, normalized by total demand
total_lead_time = grb.quicksum(lead_time[s, p] * flow_vars_ocean[s, p] for s in INTL_suppliers for p in ports) / total_demand + \
                  grb.quicksum(lead_time[p, d] * flow_vars_road[p, d] for p in ports for d in distribution_centers) / total_demand + \
                  grb.quicksum(lead_time[s, d] * flow_vars_road[s, d] for s in US_suppliers for d in distribution_centers) / total_demand

# Adding Constraints

# Ensure demand is met at each distribution center
for d, distribution_center in distribution_centers.items():
    model.addConstr(grb.quicksum(flow_vars_road[s, d] for s in US_suppliers) +
                    grb.quicksum(flow_vars_road[p, d] for p in ports) == distribution_centers[d].demand)

# Ensure supply limits are not exceeded for U.S. suppliers
for s, supplier in US_suppliers.items():
    model.addConstr(grb.quicksum(flow_vars_road[s, d] for d in distribution_centers) <= suppliers[s].supply)

# Ensure supply limits are not exceeded for international suppliers
for s, supplier in INTL_suppliers.items():
    model.addConstr(grb.quicksum(flow_vars_ocean[s, p] for p in ports) <= suppliers[s].supply)

# Ensure flow balance at ports (incoming ocean flows equal outgoing road flows)
for p, port in ports.items():
    model.addConstr(grb.quicksum(flow_vars_ocean[s, p] for s in INTL_suppliers) ==
                    grb.quicksum(flow_vars_road[p, d] for d in distribution_centers))


### **Single Objective Optimization**

***❗Task 1a: Optimize for each individual objective - cost***

Set the model's objective function to minimize total cost (in \$). Then, check the total cost value from the output of the print statement.

In [ ]:
# Your code here



print(round(total_cost.getValue(), 2))

***❗Task 1b: Optimize for each individual objective - emissions***
Set the model's objective function to minimize total emissions (in tons of CO2). Then, check the total emissions value from the output of the print statement.

In [ ]:
# Your code here



print(round(total_emissions.getValue(), 2))

***❗Task 1c: Optimize for each individual objective - lead time***
Set the model's objective function to minimize the average lead time (in days). Then, check the lead time value from the output of the print statement.

In [ ]:
# Your code here



print(round(total_lead_time.getValue(), 2))

### **Converting Objectives into a single Unit**

***❗Task 2: Converting the emissions into cost***

Define the expression for `total_cost_and_emissions`, which integrates costs and emissions by converting emissions using a social cost of \$185 per ton of CO2.  Set the model's objective function to minimize this combined value. Verify the total cost and emissions value from the output of the print statement.

In [ ]:
# Social cost of CO2 in dollars per ton, based on referenced study
social_cost_emissions = 185

# Combining the total transportation cost with the social cost of emissions

# Your code here





print(round(total_cost_and_emissions.getValue(), 2))

### **Conversion of one objective**

***❗Task 3a: Find utopia and nadir points***

Optimize for Total cost and emissions to find the utopia for Total cost and emissions and the nadir for Lead time.

In [ ]:
# Task 3.a

# Your code here





print('Utopia for Total Cost and Emissions: ', round(utopia_cost_emissions, 2))
print('Nadir for Lead Time: ', round(nadir_lead_time, 2))

***❗Task 3b: Find utopia and nadir points***

Optimize for Lead time to find the utopia for Lead time and the nadir for Total cost and emissions

In [ ]:
# Task 3.b

# Your code here





print('Utopia for Lead Time: ', round(utopia_lead_time, 2))
print('Nadir for Total Cost and Emissions: ', round(nadir_cost_emissions, 2))

***❗Task 4: Normalized values***

Define the expression of a normalized cost and emissions and normalized lead time. Then, set the models objective function to minimize the normalized cost and emissions. Verify the obtained values from the output of the print statement.

In [ ]:
# Your code here






print('Normalized Cost and Emissions: ', round(normalized_cost_emissions.getValue(), 2))
print('Total Cost and Emissions: ', round(total_cost_and_emissions.getValue(), 2))
print('Normalized Lead Time: ', round(normalized_lead_time.getValue(), 2))
print('Total Lead Time: ', round(total_lead_time.getValue(), 2))


***❗Task 5: Weighted sum method***

Implement the weighted sum method with the two objectives, `normalized_cost_emissions` and `normalized_lead_time`. Use the weights provided in the notebook to obtain the solution.

In [ ]:
weight_cost_emissions = 0.6
weight_lead_time = 0.4

# Your code here







print('Combined Objective: ', round(combined_objective.getValue(), 2))
print('Normalized Lead Time: ', round(normalized_lead_time.getValue(), 2))
print('Normalized Cost and Emissions: ', round(normalized_cost_emissions.getValue(), 2))

print("---")

print('Total Lead Time: ', round(total_lead_time.getValue(), 2))
print('Total Cost and Emissions: ', round(total_cost_and_emissions.getValue(), 2))



***❗Task 6: Epsilon constraint method***

Implement the Epsilon constraint method by setting `normalized_cost_emissions` as the primary objective and using `normalized_lead_time` as the Epsilon constraint. Ensure that `normalized_lead_time` is restricted to be less than 0.8.

In [ ]:
epsilon = 0.8

# Your code here






print('Combined Objective: ', round(combined_objective.getValue(), 2))
print('Normalized Lead Time: ', round(normalized_lead_time.getValue(), 2))
print('Normalized Cost and Emissions: ', round(normalized_cost_emissions.getValue(), 2))

print("---")

print('Total Lead Time: ', round(total_lead_time.getValue(), 2))
print('Total Cost and Emissions: ', round(total_cost_and_emissions.getValue(), 2))

